In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
num_classes = 10
num_epochs = 6
batch_size = 100
learning_rate = 0.001

In [4]:
#MNIST Dataset
train_dataset = torchvision.datasets.MNIST(root='../../data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='../../data', train=False, transform=transforms.ToTensor())

# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

卷积层维度计算:Output Size = (Input Size−Kernel Size+2×Padding)/stride + 1
池化层维度计算：Output Size = Input Size / Stride




 

In [6]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes = 10):
        super(SimpleCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.fc = nn.Linear(7 * 7 * 32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [7]:
model = SimpleCNN(num_classes).to(device)

In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/6], Step [100/600], Loss: 0.1924
Epoch [1/6], Step [200/600], Loss: 0.0741
Epoch [1/6], Step [300/600], Loss: 0.1905
Epoch [1/6], Step [400/600], Loss: 0.0201
Epoch [1/6], Step [500/600], Loss: 0.0217
Epoch [1/6], Step [600/600], Loss: 0.0404
Epoch [2/6], Step [100/600], Loss: 0.0275
Epoch [2/6], Step [200/600], Loss: 0.0655
Epoch [2/6], Step [300/600], Loss: 0.0235
Epoch [2/6], Step [400/600], Loss: 0.0383
Epoch [2/6], Step [500/600], Loss: 0.0199
Epoch [2/6], Step [600/600], Loss: 0.0666
Epoch [3/6], Step [100/600], Loss: 0.0128
Epoch [3/6], Step [200/600], Loss: 0.0305
Epoch [3/6], Step [300/600], Loss: 0.0148
Epoch [3/6], Step [400/600], Loss: 0.0176
Epoch [3/6], Step [500/600], Loss: 0.0051
Epoch [3/6], Step [600/600], Loss: 0.0099
Epoch [4/6], Step [100/600], Loss: 0.0169
Epoch [4/6], Step [200/600], Loss: 0.0295
Epoch [4/6], Step [300/600], Loss: 0.0141
Epoch [4/6], Step [400/600], Loss: 0.1414
Epoch [4/6], Step [500/600], Loss: 0.0026
Epoch [4/6], Step [600/600], Loss:

In [10]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98.77 %
